In [36]:
from collections.abc import Callable
from functools import reduce

Operation = Callable[[int], int]
Test = Callable[[int], bool]

class Item:
    def __init__(self, worry: int, should_settle: bool):
        self.worry = worry
        self.should_settle = should_settle

    def __str__(self):
        return str(self.worry)
    
    def settle_after_inspect(self):
        if (self.should_settle): self.worry = self.worry // 3

class Monkey:
    def __init__(self, name: str, items: list[Item], op: Operation, test_num: int, get_true_monkey, get_false_monkey):
        self.name = name
        self.items = items
        self.op = op
        self.test_num = test_num
        self.get_true_monkey = get_true_monkey
        self.get_false_monkey = get_false_monkey
        self.items_inspected = 0
    
    def realize_next_monkeys(self):
        self.true_monkey = self.get_true_monkey()
        self.false_monkey = self.get_false_monkey()
    
    def run_round(self, round_num: int, verbose: bool):
        # if verbose: print(f'{self.name} starts {round_num}:')

        for item in self.items:
            # if verbose: print(f'    {self.name} inspects item with worry {item.worry}')
            new_worry = self.op(item.worry)
            # if verbose: print(f'    Inspection changes worry to {new_worry}')
            item.worry = new_worry
            item.settle_after_inspect()
            # if verbose: print(f'    Monkey gets bored and worry settles to {item.worry}')
            test_result = item.worry % self.test_num == 0
            if test_result:
                #item.worry = item.worry / self.test_num
                self.true_monkey.recieve_item(item)
            else:
                self.false_monkey.recieve_item(item)
            # if verbose: print(f'    Item test is {test_result} tossing to next monkey {next_monkey.name}')
            self.items_inspected += 1
        self.items = []

    def recieve_item(self, item: Item):
        self.items.append(item)

def get_monkey_getter(monkey_list:list[Monkey], monkey_num:int):
    def monkey_getter():
        return monkey_list[monkey_num]
    return monkey_getter

def get_operation_doer(op_str: str):
    compiled = eval(f'lambda old: {op_str}')
    return compiled

def get_tester(test_val: int):
    def do_test(val: int):
        return val % test_val == 0
    return do_test

def parse_notes(notes: str, should_settle: bool):
    monkey_list:list[Monkey] = []
    all_items:list[Item] = []
    per_monkey = notes.split('\n\n')
    for monk in per_monkey:
        monk_parts = monk.splitlines()
        name = monk_parts[0].strip(':')
        items = [Item(int(val), should_settle) for val in monk_parts[1].split(': ')[1].split(', ')]
        all_items.extend(items)
        op_str = monk_parts[2].split('= ')[1]
        test_val = int(monk_parts[3].split('divisible by ')[1])
        true_monkey = int(monk_parts[4].split('monkey ')[1])
        false_monkey = int(monk_parts[5].split('monkey ')[1])

        monkey_list.append(Monkey(
            name,
            items,
            get_operation_doer(op_str),
            test_val,
            get_monkey_getter(monkey_list, true_monkey),
            get_monkey_getter(monkey_list, false_monkey)
        ))
    for monkey in monkey_list:
        monkey.realize_next_monkeys()
    return (monkey_list, all_items)

def do_rounds(monkey_list:list[Monkey], num_rounds: int):
    for round in range(num_rounds):
        for monkey in monkey_list:
            monkey.run_round(round, False)
    
    return sorted(monkey_list, key=lambda x: x.items_inspected, reverse=True)


In [37]:
notes = open('./data.txt', 'r').read()

(monkey_list, all_items) = parse_notes(notes, True)

done = do_rounds(monkey_list, 20)

mb = done[0].items_inspected * done[1].items_inspected
mb

107822

In [80]:
audit: list[str] = []

class ModuloRecord:
    def __init__(self, initial_value: int, divisor: int):
        self.initial_value = initial_value
        self.divisor = divisor
        self.modulo = initial_value % divisor
    
    def mult(self, mult: int):
        self.modulo = (self.modulo * mult) % self.divisor
    
    def add(self, addend: int):
        self.modulo = (self.modulo + addend) % self.divisor
    
    def square(self):
        inv = self.divisor - self.modulo
        self.modulo = (inv ** 2) % self.divisor

class ModuloItem:
    def __init__(self, initial_value: int):
        self.initial_value = initial_value
        self.modulos: dict[str, ModuloRecord] = {}
    
    def __str__(self):
        values = []
        for div, item in self.modulos.items():
            values.append(f'{div}: {item.modulo}')
        return f'{self.initial_value} [{" ".join(values)}]'
    
    def calculate_modulos(self, divisors: list[int]):
        for divisor in divisors:
            self.modulos[str(divisor)] = ModuloRecord(self.initial_value, divisor)
    
    def test(self, divisor: int):
        global audit
        record = self.modulos[str(divisor)]
        result = record.modulo == 0
        # audit.append(f'test {self.initial_value} against {divisor} {result}')
        # audit.append(str(self))
        return result
    
    def square(self):
        global audit
        for div, item in self.modulos.items():
            item.square()
        # audit.append(f'square {self.initial_value}')
        # audit.append(str(self))
    
    def mult(self, mult: int):
        global audit
        for div, item in self.modulos.items():
            item.mult(mult)
        # audit.append(f'mult {mult} by {self.initial_value}')
        # audit.append(str(self))
    
    def add(self, addend: int):
        global audit
        for div, item in self.modulos.items():
            item.add(addend)
        # audit.append(f'add {addend} to {self.initial_value}')
        # audit.append(str(self))

ModuloOperation = Callable[[ModuloItem], None]

class ModuloMonkey:
    def __init__(self, name: str, items: list[ModuloItem], op: ModuloOperation, test_num: int, get_true_monkey, get_false_monkey):
        self.name = name
        self.items = items
        self.op = op
        self.test_num = test_num
        self.get_true_monkey = get_true_monkey
        self.get_false_monkey = get_false_monkey
        self.items_inspected = 0
    
    def realize_next_monkeys(self):
        self.true_monkey = self.get_true_monkey()
        self.false_monkey = self.get_false_monkey()
    
    def run_round(self):
        for item in self.items:
            self.op(item)
            if item.test(self.test_num):
                self.true_monkey.recieve_item(item)
            else:
                self.false_monkey.recieve_item(item)
            self.items_inspected += 1
        self.items = []

    def recieve_item(self, item: Item):
        self.items.append(item)

def get_modulo_operation_doer(op_parts: list[str]):
    if (op_parts[0] == 'old' and op_parts[2] == 'old'):
        def square(item: ModuloItem): item.square()
        return square
    else:
        val = int(op_parts[2])
        if op_parts[1] == '*':
            def mult(item: ModuloItem): item.mult(val)
            return mult
        else:
            def add(item:ModuloItem): item.add(val)
            return add

def parse_modulo_notes(notes: str):
    monkey_list:list[ModuloMonkey] = []
    all_items:list[ModuloItem] = []
    divisors: list[int] = []
    per_monkey = notes.split('\n\n')
    for monk in per_monkey:
        monk_parts = monk.splitlines()
        name = monk_parts[0].strip(':')
        items = [ModuloItem(int(val)) for val in monk_parts[1].split(': ')[1].split(', ')]
        all_items.extend(items)
        op_parts = monk_parts[2].split('= ')[1].split(' ')
        test_val = int(monk_parts[3].split('divisible by ')[1])
        divisors.append(test_val)
        true_monkey = int(monk_parts[4].split('monkey ')[1])
        false_monkey = int(monk_parts[5].split('monkey ')[1])

        monkey_list.append(ModuloMonkey(
            name,
            items,
            get_modulo_operation_doer(op_parts),
            test_val,
            get_monkey_getter(monkey_list, true_monkey),
            get_monkey_getter(monkey_list, false_monkey)
        ))
    for monkey in monkey_list:
        monkey.realize_next_monkeys()
    
    for item in all_items:
        item.calculate_modulos(divisors)
        audit.append(str(item))
    return (monkey_list, all_items)

def do_modulo_rounds(monkey_list:list[ModuloMonkey], num_rounds: int):
    for round in range(num_rounds):
        for monkey in monkey_list:
            monkey.run_round()
    
    return sorted(monkey_list, key=lambda x: x.items_inspected, reverse=True)

In [82]:
notes = open('./data.txt', 'r').read()

(monkey_list_2, all_items2) = parse_modulo_notes(notes)

done = do_modulo_rounds(monkey_list_2, 10000)

# print([len(monkey.items) for monkey in monkey_list_2])

mb2 = done[0].items_inspected * done[1].items_inspected
mb2

27267163742

In [60]:
val = 46
div = 17
print(val % div, (val * val) % div)

# (50 * 50) % 17
# (16 * 3) % 17

12 8
